# Bonsai in Tensorflow

This is a simple notebook that illustrates the usage of Tensorflow implementation of Bonsai. We are using the USPS dataset. Please refer to `fetch_usps.py` and run it for downloading and cleaning up the dataset.

In [1]:
# Copyright (c) Microsoft Corporation. All rights reserved.
# Licensed under the MIT license.

import helpermethods
import tensorflow as tf
import numpy as np
import sys
import os

#Provide the GPU number to be used
os.environ['CUDA_VISIBLE_DEVICES'] =''

#Bonsai imports
from edgeml.trainer.bonsaiTrainer import BonsaiTrainer
from edgeml.graph.bonsai import Bonsai

# Fixing seeds for reproducibility
tf.set_random_seed(42)
np.random.seed(42)

# USPS Data

It is assumed that the USPS data has already been downloaded and set up with the help of [fetch_usps.py](fetch_usps.py) and is present in the `./usps10` subdirectory.

In [2]:
#Loading and Pre-processing dataset for Bonsai
dataDir = "usps10/"
(dataDimension, numClasses, Xtrain, Ytrain, Xtest, Ytest, mean, std) = helpermethods.preProcessData(dataDir, isRegression=False)
print("Feature Dimension: ", dataDimension)
print("Num classes: ", numClasses)

Feature Dimension:  257
Num classes:  10


# Model Parameters

Note that Bonsai is designed for low-memory setting and the best results are obtained when operating in that setting. Use the sparsity, projection dimension and tree depth to vary the model size.

In [3]:
sigma = 1.0 #Sigmoid parameter for tanh
depth = 3 #Depth of Bonsai Tree
projectionDimension = 28 #Lower Dimensional space for Bonsai to work on

#Regularizers for Bonsai Parameters
regZ = 0.0001
regW = 0.001
regV = 0.001
regT = 0.001

totalEpochs = 100

learningRate = 0.01

outFile = None

#Sparsity for Bonsai Parameters. x => 100*x % are non-zeros
sparZ = 0.2
sparW = 0.3
sparV = 0.3
sparT = 0.62

batchSize = np.maximum(100, int(np.ceil(np.sqrt(Ytrain.shape[0]))))

useMCHLoss = True #only for Multiclass cases True: Multiclass-Hing Loss, False: Cross Entropy. 

#Bonsai uses one classier for Binary, thus this condition
if numClasses == 2:
    numClasses = 1

Placeholders for Data feeding during training and infernece

In [4]:
X = tf.placeholder("float32", [None, dataDimension])
Y = tf.placeholder("float32", [None, numClasses])

Creating a directory for current model in the datadirectory using timestamp

In [5]:
currDir = helpermethods.createTimeStampDir(dataDir)
helpermethods.dumpCommand(sys.argv, currDir)

# Bonsai Graph Object

Instantiating the Bonsai Graph which will be used for training and inference.

In [6]:
bonsaiObj = Bonsai(numClasses, dataDimension, projectionDimension, depth, sigma)

# Bonsai Trainer Object

Instantiating the Bonsai Trainer which will be used for 3 phase training.

In [7]:
bonsaiTrainer = BonsaiTrainer(bonsaiObj, regW, regT, regV, regZ, sparW, sparT, sparV, sparZ,
                              learningRate, X, Y, useMCHLoss, outFile)

C:\Users\t-vekusu\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\ops\gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Session declaration and variable initialization. 
Interactive Session doesn't clog the entire GPU.

In [8]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

# Bonsai Training Routine

The method to to run the 3 phase training, followed by giving out the best early stopping model, accuracy along with saving of the parameters.

In [9]:
bonsaiTrainer.train(batchSize, totalEpochs, sess,
                    Xtrain, Xtest, Ytrain, Ytest, dataDir, currDir)


Epoch Number: 0

******************** Dense Training Phase Started ********************


Classification Train Loss: 6.388934433460236
Training accuracy (Classification): 0.6250000005174015
Test accuracy 0.726956
MarginLoss + RegLoss: 1.4466879 + 3.6487768 = 5.0954647


Epoch Number: 1

Classification Train Loss: 3.6885906954606376
Training accuracy (Classification): 0.8623611107468605
Test accuracy 0.758346
MarginLoss + RegLoss: 1.0173264 + 2.778634 = 3.7959604


Epoch Number: 2

Classification Train Loss: 2.667721450328827
Training accuracy (Classification): 0.9184722271230485
Test accuracy 0.7429
MarginLoss + RegLoss: 0.92546654 + 2.095467 = 3.0209336


Epoch Number: 3

Classification Train Loss: 1.9921080254846149
Training accuracy (Classification): 0.941944446000788
Test accuracy 0.767314
MarginLoss + RegLoss: 0.7603649 + 1.5889603 = 2.3493252


Epoch Number: 4

Classification Train Loss: 1.5233625107341342
Training accuracy (Classification): 0.9563888907432556
Test accuracy 0.79


Epoch Number: 40

Classification Train Loss: 0.17237282171845436
Training accuracy (Classification): 0.9837500088744693
Test accuracy 0.937718
MarginLoss + RegLoss: 0.17473482 + 0.11479883 = 0.28953364


Epoch Number: 41

Classification Train Loss: 0.16901198805620274
Training accuracy (Classification): 0.9837500097023116
Test accuracy 0.93423
MarginLoss + RegLoss: 0.17860568 + 0.112817116 = 0.29142278


Epoch Number: 42

Classification Train Loss: 0.16710670509686074
Training accuracy (Classification): 0.9833333442608515
Test accuracy 0.936722
MarginLoss + RegLoss: 0.17501548 + 0.11118551 = 0.286201


Epoch Number: 43

Classification Train Loss: 0.16463725310232905
Training accuracy (Classification): 0.9836111209458775
Test accuracy 0.93423
MarginLoss + RegLoss: 0.17687047 + 0.10897398 = 0.28584445


Epoch Number: 44

Classification Train Loss: 0.16215091271118987
Training accuracy (Classification): 0.9843055663837327
Test accuracy 0.935227
MarginLoss + RegLoss: 0.17832607 + 0.107886


Epoch Number: 80

Classification Train Loss: 0.14349345521380505
Training accuracy (Classification): 0.9861111185616918
Test accuracy 0.941206
MarginLoss + RegLoss: 0.16280341 + 0.09526416 = 0.25806758


Epoch Number: 81

Classification Train Loss: 0.14298133655554718
Training accuracy (Classification): 0.9848611205816269
Test accuracy 0.935725
MarginLoss + RegLoss: 0.16992427 + 0.095204785 = 0.26512906


Epoch Number: 82

Classification Train Loss: 0.1410345918395453
Training accuracy (Classification): 0.9854166756073633
Test accuracy 0.937718
MarginLoss + RegLoss: 0.16711517 + 0.09361006 = 0.26072523


Epoch Number: 83

Classification Train Loss: 0.14173460192978382
Training accuracy (Classification): 0.9858333418766657
Test accuracy 0.935227
MarginLoss + RegLoss: 0.17255607 + 0.09335034 = 0.26590642


Epoch Number: 84

Classification Train Loss: 0.1413275660533044
Training accuracy (Classification): 0.985000009338061
Test accuracy 0.939213
MarginLoss + RegLoss: 0.1691187 + 0.092208